# Align Data Cube

The raw data of the FISS are rotated with time because the FISS does not have the derotator. In addition, an object can be shifted by the effect of the seeing, and the image itself is reversed by the mirror. To correct these effects, we should align the data. Using the cross-correlation technique between two images and the interpolation, we can successfully align the FISS data. Here, we show how to co-align the data obatined by two cameras. First, we calculate the align parameters, such as a rotating angle, and shift in x, and y directions using the [calAlignPars](../#calAlignPars), then we co-align the image of camera A with that of camera B using the [alignCams](../#alignCams). These two procedures are combined with the [alignAll](../#alignAll) function, so you can simply align the data like this:

In [1]:
import fisspy
from glob import glob
from os.path import join
import numpy as np

bdir = r'D:\Data\140603'
lfa = glob(join(bdir,'*A1_c.fts'))# list of the file obtained by the camera A
lfb = glob(join(bdir,'*B1_c.fts'))# list of the file obtained by the camera B
lfa.sort() # sorting in time sequence
lfb.sort() # sorting in time sequence
apA, apB = fisspy.align.alignAll(lfa, lfb, sname=join(bdir,'alignpar_A.npz'))

Align cam A.
Running Alignment
Done        
Align cam B.
Running Alignment
Done        
Align two cameras
Write alignpar: D:\Data\140603\alignpar_A.npz .
Write alignpar: D:\Data\140603\alignpar_B.npz .


It will save the two align parameter files for each camera. Note that if there is any abnormal data, such as unlocked data or largely shifted data, it cannot successfully align the whole data. Therefore, you should check whether the data has such abnormal data in the series.

Next step is making the data cube using these align parameters. For this we use [alignDataCube](../#alignDataCube) for only one data set or [alignTwoDataCubes](../#alignTwoDataCubes) for two cameras. Here, we show the latter case.

In [2]:
nf = len(lfa)
refA = fisspy.read.FISS(lfa[nf//2])
refB = fisspy.read.FISS(lfb[nf//2])
nxA = refA.nx
nyA = refA.ny
nxB = refB.nx
nyB = refB.ny

alignA = np.zeros((nf,nyA,nxA), dtype=float)
alignB = np.zeros((nf,nyB,nxB), dtype=float)

print('Running make cube') 
print('    0 %', end='\r', flush=True) #to check the running status

# make unaligned time series data.
for i, f in enumerate(lfa):
    print(f'    {(i+1)*100/nf:.2f} %', end='\r', flush=True)
    fiss = fisspy.read.FISS(f, wvCalibMethod='photo')
    fissB = fisspy.read.FISS(lfb[i], wvCalibMethod='photo')
    fiss.lambdameter(hw=0.2)
    fissB.lambdameter(hw=0.1)
    alignA[i] = fiss.lv
    alignB[i] = fissB.lv

print('Done             ')

fapA = join(bdir, "alignpar_A.npz")
fapB = join(bdir, "alignpar_B.npz")

avA, avB = fisspy.align.alignTwoDataCubes(alignA, alignB, fapA, fapB)  # make aligned data cube

apA = fisspy.align.readAlignPars(fapA)
fisspy.align.saveAlignCube(avA, apA['time'], sname=join(bdir, '140603_alosv_A.npz')) # save the aligned data cube to read the data using the alignCube code.
apB = fisspy.align.readAlignPars(fapB)
fisspy.align.saveAlignCube(avB, apB['time'], sname=join(bdir, '140603_alosv_B.npz')) # save the aligned data cube to read the data using the alignCube code.

Running make cube
Done             


Concuraturation!! You can successfully align the data!! To check the aligned data cube, please use [alignCube](/fisspy/read/alignCube) module in fisspy.read.